<a href="https://colab.research.google.com/github/doanane/-_-/blob/main/sentimental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Context
This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment .

Content
It contains the following 6 fields:

1. target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

2. ids: The id of the tweet ( 2087)

3. date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

4. flag: The query (lyx). If there is no query, then this value is NO_QUERY.

5. user: the user that tweeted (robotickilldozr)

6. text: the text of the tweet (Lyx is cool)

### importing twitter sentiment analysis dataset

In [ ]:
##using kaggle api

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


Extracting the compressed dataset

In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip' #path for our zip, it was imported from our local disk
with ZipFile(dataset,'r') as zip:  # r will read the dataset name given and it is reading it as zip a name we chose
  zip.extractall() #extact all the file and give as a feedback it it was extracted successfuly
  print('The dataset is extracted')  # when successful we will check if we had a train...csv file



The dataset is extracted


In [ ]:
## importing our libraries
import numpy as np
import pandas as pd  ## use to create our datafrome
import matplotlib.pyplot as plt
import seaborn as sns
import re  ## regular expression, used in order to do some pattern ,maching
import string  ## to work with strings
import nltk ## natural  language used in our textual data
from nltk.corpus import stopwords ## to remove stopwords
from nltk.stem.porter import PorterStemmer ## used to reduce a word to its root
from sklearn.feature_extraction.text import TfidfVectorizer ## used to convert textual data to numerical data
from sklearn.model_selection import train_test_split ## used to split our data into train and test
from sklearn.linear_model import LogisticRegression ## used to train our model
from sklearn.metrics import accuracy_score  ## used to evaluate our model
from sklearn.metrics import classification_report


In [ ]:
nltk.download('stopwords') ## downloading stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
## printing stopwords in english
stopwords.words('english') ## doesn't add any meanining

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
 # loading our data from the csv file to pandas dataframe
elon_musk = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='latin-1',header=None)
elon_musk.shape # checking the number of rule and colunms for verification


(1600000, 6)

In [ ]:
elon_musk.head() # checking our first five tweets

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# making our columns more readable
colunm_names = ['target','ids','date','flag','user','text']
elon_musk.columns = colunm_names
elon_musk.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Checking for missing value by either replacing it or dropping it
elon_musk.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
#checking the distribution column, thats checking for positive and negative tweets
elon_musk['target'].value_counts()

,count
target,
0,800000
4,800000


We  want to change our target definitions, that we want to change the initial definition of the dataset to our own terms by representing:
 0 = negative tweet and
 1 = positive tweet

In [ ]:
elon_musk.replace({'target':{4:1}},inplace=True)
elon_musk.replace({'target':{"negative":0}},inplace=True)
pd.set_option('future.no_silent_downcasting', True)
elon_musk['target'].value_counts()


,count
target,
0,800000
1,800000


**Steming**

the imported library PorterSteamer will help reduce a word to its **keyword or root word**
example: acting,actor, actress = act

In [ ]:
porter_stemmer = PorterStemmer()


In [ ]:
# This will load the instace of the PotterSteamer function
# def stemming(content): #when this is called we want to reduce words to its keyword
#   stemmed_content = re.sub('[^a-zA-Z]',' ',content)  # considering only alphabet
#   stemmed_content = stemmed_content.lower() ## used for converting uppercase to lowercase
#   stemmed_content = stemmed.split() #split the words from the tweet
#   stemmed_content = [porter_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
#   stemmed_content = ' '.join(stemmed_content)
#   return stemmed_content
#
pattern =re.compile('[^a-zA-Z]')
english_stopwords = stopwords.words('english')
port_stemmer = PorterStemmer()

def stemming(content):
  stemmed_content = re.sub(pattern,' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  return ' '.join([port_stemmer.stem(word) for word in stemmed_content if not word in english_stopwords])


**alternaive**
the code below takes about 50 minutes to execute so we prefered the one above since it took about 6 minute to execute

In [ ]:
def stemming(content): #when this is called we want to reduce words to its keyword
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)  # considering only alphabet
  stemmed_content = stemmed_content.lower() ## used for converting uppercase to lowercase
  stemmed_content = stemmed_content.split() #split the words from the tweet
  stemmed_content = [porter_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
elon_musk['stemmed_content'] = elon_musk['text'].apply(stemming)

In [ ]:
elon_musk.head()


,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
elon_musk['stemmed_content']

,stemmed_content
0,switchfoot http twitpic com zl awww bummer sho...
1,upset updat facebook text might cri result sch...
2,kenichan dive mani time ball manag save rest g...
3,whole bodi feel itchi like fire
4,nationwideclass behav mad see
...,...
1599995,woke school best feel ever
1599996,thewdb com cool hear old walt interview http b...
1599997,readi mojo makeov ask detail
1599998,happi th birthday boo alll time tupac amaru sh...


In [ ]:
elon_musk['target']

,target
0,0
1,0
2,0
3,0
4,0
...,...
1599995,1
1599996,1
1599997,1
1599998,1


In [ ]:
# Seperating the data and label (tweet and target)
tweet = elon_musk['stemmed_content'].values
target = elon_musk['target'].values

In [ ]:
print(tweet)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(target)

[0 0 0 ... 1 1 1]


In [ ]:
# splitting the data into training data and test data

**Splitting our data into testing and training data**

In [ ]:
tweet_train, tweet_test, target_train, target_test = train_test_split(tweet,target,test_size=0.25,stratify=target,random_state=2)
# stratified distribute equal proportion of 0 and 1

In [ ]:
print(tweet.shape, tweet_train.shape, tweet_test.shape)


(1600000,) (1200000,) (400000,)


**Feature extraction**

```
converting textual data to numeric data
```



In [ ]:
# vectorizer = TfidfVectorizer()
# vectorizer.fit(tweet_train)

# tweet_train = vectorizer.transform(tweet_train)
# tweet_test = vectorizer.transform(tweet_test)

vectorizer = TfidfVectorizer()

tweet_train = vectorizer.fit_transform(tweet_train)
tweet_test = vectorizer.transform(tweet_test)

In [ ]:
print(type(tweet_train))


<class 'scipy.sparse._csr.csr_matrix'>


In [ ]:
print(tweet_train)

  (0, 365189)	0.5132080660144132
  (0, 237343)	0.23303345421625404
  (0, 374356)	0.448146199728837
  (0, 403997)	0.24502287044549792
  (0, 76067)	0.26933867919175003
  (0, 347166)	0.28801409672291506
  (0, 353480)	0.31024983563960834
  (0, 173944)	0.27361083420633164
  (0, 76623)	0.3079421596528722
  (1, 75463)	0.20046450884815897
  (1, 327520)	0.5889858120350425
  (1, 74743)	0.26384919818910885
  (1, 417988)	0.34047165848158994
  (1, 372368)	0.256832440572957
  (1, 418472)	0.3323599211136618
  (1, 161685)	0.1989697793628871
  (1, 27594)	0.24897486619137327
  (1, 102200)	0.23313119944256042
  (1, 232174)	0.1701036662137205
  (1, 419352)	0.2570670225806945
  (2, 241943)	0.4596298904469063
  (2, 120272)	0.2381823821827771
  (2, 276774)	0.2119405837064331
  (2, 89868)	0.14578917547323808
  (2, 389393)	0.17351818363296903
  :	:
  (1199995, 28119)	0.3431722253942965
  (1199995, 380922)	0.3341270409757624
  (1199995, 441344)	0.3842355334804584
  (1199995, 287685)	0.441029665732584
  (1199995

In [ ]:
print(tweet_train)

  (0, 365189)	0.5132080660144132
  (0, 237343)	0.23303345421625404
  (0, 374356)	0.448146199728837
  (0, 403997)	0.24502287044549792
  (0, 76067)	0.26933867919175003
  (0, 347166)	0.28801409672291506
  (0, 353480)	0.31024983563960834
  (0, 173944)	0.27361083420633164
  (0, 76623)	0.3079421596528722
  (1, 75463)	0.20046450884815897
  (1, 327520)	0.5889858120350425
  (1, 74743)	0.26384919818910885
  (1, 417988)	0.34047165848158994
  (1, 372368)	0.256832440572957
  (1, 418472)	0.3323599211136618
  (1, 161685)	0.1989697793628871
  (1, 27594)	0.24897486619137327
  (1, 102200)	0.23313119944256042
  (1, 232174)	0.1701036662137205
  (1, 419352)	0.2570670225806945
  (2, 241943)	0.4596298904469063
  (2, 120272)	0.2381823821827771
  (2, 276774)	0.2119405837064331
  (2, 89868)	0.14578917547323808
  (2, 389393)	0.17351818363296903
  :	:
  (1199995, 28119)	0.3431722253942965
  (1199995, 380922)	0.3341270409757624
  (1199995, 441344)	0.3842355334804584
  (1199995, 287685)	0.441029665732584
  (1199995

In [ ]:
print(tweet_test)

  (0, 2660)	0.3601783047868634
  (0, 29794)	0.2065818431550562
  (0, 83586)	0.46809413234577185
  (0, 142254)	0.2816038871445881
  (0, 153988)	0.24966370327216753
  (0, 155084)	0.26850580709852695
  (0, 162747)	0.34342514882547964
  (0, 165295)	0.27283113951697135
  (0, 288731)	0.279794468955125
  (0, 396888)	0.3522679708361543
  (1, 89868)	0.1629004007027813
  (1, 119508)	0.3338431294300748
  (1, 127362)	0.241680550521661
  (1, 143622)	0.1524821950529528
  (1, 154194)	0.3619263202636623
  (1, 157078)	0.6215471764912583
  (1, 267393)	0.20801913348714463
  (1, 421479)	0.3188692473686777
  (1, 427020)	0.34364342763470335
  (2, 6335)	0.34105108112357657
  (2, 89868)	0.16516525262129028
  (2, 128773)	0.359508435772894
  (2, 153299)	0.5512532916983284
  (2, 173871)	0.300877343850007
  (2, 225144)	0.1760051569672463
  :	:
  (399996, 61399)	0.6082387400738714
  (399996, 144467)	0.24081453257477542
  (399996, 241630)	0.3775126043628339
  (399996, 267912)	0.3309813811971613
  (399996, 394299)	0

In [ ]:
print(tweet_test)

  (0, 2660)	0.3601783047868634
  (0, 29794)	0.2065818431550562
  (0, 83586)	0.46809413234577185
  (0, 142254)	0.2816038871445881
  (0, 153988)	0.24966370327216753
  (0, 155084)	0.26850580709852695
  (0, 162747)	0.34342514882547964
  (0, 165295)	0.27283113951697135
  (0, 288731)	0.279794468955125
  (0, 396888)	0.3522679708361543
  (1, 89868)	0.1629004007027813
  (1, 119508)	0.3338431294300748
  (1, 127362)	0.241680550521661
  (1, 143622)	0.1524821950529528
  (1, 154194)	0.3619263202636623
  (1, 157078)	0.6215471764912583
  (1, 267393)	0.20801913348714463
  (1, 421479)	0.3188692473686777
  (1, 427020)	0.34364342763470335
  (2, 6335)	0.34105108112357657
  (2, 89868)	0.16516525262129028
  (2, 128773)	0.359508435772894
  (2, 153299)	0.5512532916983284
  (2, 173871)	0.300877343850007
  (2, 225144)	0.1760051569672463
  :	:
  (399996, 61399)	0.6082387400738714
  (399996, 144467)	0.24081453257477542
  (399996, 241630)	0.3775126043628339
  (399996, 267912)	0.3309813811971613
  (399996, 394299)	0

Training our Machine Learning model using logistic regression

In [ ]:
model = LogisticRegression(max_iter=1000)  #model can go trough the data 1000 times

In [ ]:
model.fit(tweet_train,target_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

In [ ]:
## making prediction from the tweet_train
accuracy_on_training_data = model.score(tweet_train,target_train)
tweet_train_prediction = model.predict(tweet_train)
training_data_accuracy = accuracy_score(tweet_train_prediction,target_train)

In [ ]:
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.797375


In [ ]:
# accuracy score on the test data

In [ ]:
tweet_test_prediction = model.predict(tweet_test)  # using the models to predict the labels for the test
test_data_accuracy = accuracy_score(tweet_test_prediction, target_test) # after all we compare the test data prediction with the true test labels which is y test

In [ ]:
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.7762475


In [ ]:
## the training data accuracyand the test data accuracy is inline since its close to each other
## if it had being 40% or 50% it would be overfitting

Reason for calculating for test data accuracy is that,
 consider the following example
 say we have been asked to study 100 questions for our DCIT313 course and out of it,
 we would be getting some of the questions in the trial question
 when we study this well we would have a good score an also
 what if the question we got in the examination is diferent from the trial question but based on the trial question we will be able to do it if and only if we have study the trial question well


Trainin data is like the on which the model has studied (trial questions we have studied and practiced) while the test data is the question we don't know but its based on the trial question

In [ ]:
## Model accuracy = 77.8% > 75%, say it was to be a lower accuracy we could have used techniques like grid sear CV, randomizer CV in order to increase accuracy

Saving the trained model

In [ ]:
import pickle as pk


In [ ]:
filename = 'trained_model.sav'
pk.dump(model,open(filename,'wb'))  # all the parameters learned by the model would be save in the filename we've created
# wb means we are only writing in binary
# therefore we don't need to train it again

Using the saved files to make our prediction

In [ ]:
# loading the saved model
golden = pk.load(open('/content/trained_model.sav','rb'))
# rb means read binary

In [ ]:
tweet_new = tweet_test[200]  # test the model hasn't seen before
print(target_test[200])

prediction = golden.predict(tweet_new)
print(prediction)

if (prediction[0]==0):
  print('The tweet is negative')
else:
  print('The tweet is positive')

1
[1]
The tweet is positive


In [ ]:
tweet_new = tweet_test[3]  # test the model hasn't seen before
print(target_test[3])

prediction = golden.predict(tweet_new)
print(prediction)

if (prediction[0]==0):
  print('The tweet is negative')
else:
  print('The tweet is positive')

0
[1]
The tweet is positive


In [ ]:
tweet_new = tweet_test[20]  # test the model hasn't seen before
print(target_test[20])

prediction = golden.predict(tweet_new)
print(prediction)

if (prediction[0]==0):
  print('The tweet is negative')
else:
  print('The tweet is positive')

0
[1]
The tweet is positive


In [ ]:
tweet_new = tweet_test[150]  # test the model hasn't seen before
print(target_test[150])

prediction = golden.predict(tweet_new)
print(prediction)

if (prediction[0]==0):
  print('The tweet is negative')
else:
  print('The tweet is positive')

0
[0]
The tweet is negative


In [ ]:
tweet_new = tweet_test[500]  # test the model hasn't seen before
print(target_test[500])

prediction = golden.predict(tweet_new)
print(prediction)

if (prediction[0]==0):
  print('The tweet is negative')
else:
  print('The tweet is positive')

1
[1]
The tweet is positive


In [ ]:
from sklearn.linear_model import LogisticRegression

# Training the logistic regression model
model = LogisticRegression(max_iter=200)
model.fit(tweet_train, target_train)


LogisticRegression(max_iter=200)

In [ ]:
def predict_sentiment(user_input, model, vectorizer):
    # Preprocess the user input
    preprocessed_input = stemming(user_input)
    # Transform the input into numerical form
    vectorized_input = vectorizer.transform([preprocessed_input])
    # Make a prediction
    prediction = model.predict(vectorized_input)
    # Interpret the prediction
    sentiment = "Positive" if prediction == 1 else "Negative"
    return sentiment


In [ ]:
# Example of user input
user_tweet = input("Enter a tweet or comment: ")

# Predict sentiment
sentiment = predict_sentiment(user_tweet, model, vectorizer)
print(f"The sentiment of the provided tweet is: {sentiment}")


Enter a tweet or comment: he is a thief
The sentiment of the provided tweet is: Negative


In [ ]:
# Example of user input
user_tweet = input("Enter a tweet or comment: ")

# Predict sentiment
sentiment = predict_sentiment(user_tweet, model, vectorizer)
print(f"The sentiment of the provided tweet is: {sentiment}")


Enter a tweet or comment: yes
The sentiment of the provided tweet is: Positive


In [ ]:
# Example of user input
user_tweet = input("Enter a tweet or comment: ")

# Predict sentiment
sentiment = predict_sentiment(user_tweet, model, vectorizer)
print(f"The sentiment of the provided tweet is: {sentiment}")


Enter a tweet or comment: Twitter is fun
The sentiment of the provided tweet is: Positive


In [ ]:
# Example of user input
user_tweet = input("Enter a tweet or comment: ")

# Predict sentiment
sentiment = predict_sentiment(user_tweet, model, vectorizer)
print(f"The sentiment of the provided tweet is: {sentiment}")


Enter a tweet or comment: Bawumia is a Digitalized man
The sentiment of the provided tweet is: Positive


In [ ]:
# Example of user input
user_tweet = input("Enter a tweet or comment: ")

# Predict sentiment
sentiment = predict_sentiment(user_tweet, model, vectorizer)
print(f"The sentiment of the provided tweet is: {sentiment}")


Enter a tweet or comment: John Mahama is the president of Ghana
The sentiment of the provided tweet is: Positive


In [ ]:
# Example of user input
user_tweet = input("Enter a tweet or comment: ")

# Predict sentiment
sentiment = predict_sentiment(user_tweet, model, vectorizer)
print(f"The sentiment of the provided tweet is: {sentiment}")


Enter a tweet or comment: Good is good
The sentiment of the provided tweet is: Positive


In [249]:
# Example of user input
user_tweet = input("Enter a tweet or comment: ")

# Predict sentiment
sentiment = predict_sentiment(user_tweet, model, vectorizer)
print(f"The sentiment of the provided tweet is: {sentiment}")


Enter a tweet or comment: Ghana is in Asia
The sentiment of the provided tweet is: Positive
